<a href="https://colab.research.google.com/github/Yuni-J/dataKakao/blob/main/%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 01 카카오톡 데이터 전처러


##구글 드라이브와 코랩 연결하기 (구글 드라이브 마운트)

In [74]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import pandas as pd

## 폴더 경로
f_path = '/content/drive/MyDrive/Katalk_data_analysis/KakaoTalk_data'

## 파일 경로
files = {
    'katalk_friend.txt'
    'katalk_oh.txt'
}

##정규 표현식 사용

\-+ d{4}년 \d{1,2}월 \d{1,2}일 [월화수목금토일]요일 -+

- -+ : “-”가 하나 이상 반복  (앞뒤의 긴 하이픈)
- \d{4}년 : 연도 (4자리 숫자 + “년)
- \d{1,2}월 : 월 (1~2자리 숫자 + “월”)
- \d{1,2}일 : 일 (1~2자리 숫자 + “일”)
- -+ : "-"가 하나 이상 반복 (뒤쪽 하이픈)


\[\S+\] \[오\S+ \d{1,2}:\d{2}\]\s+(.+)

- \[\S+\]  : 대괄호 안에 공백이 없는 문자열
- \[오\S+ \d{1,2}:\d{2}\]\s :
- - 오\S+ : “오”가 들어가는 공백이 없는 문자열
- - 공백
- -  \d{1,2}:\d{2} : 시간 형식
- (.+) : 내용 추출


In [ ]:
import re

text = """--------------- 2022년 1월 16일 일요일 ---------------
[민지이잉] [오전 12:02] ㅋㅋㅋㅋㅋㅋㅋㅋㄱㅇㄱㅋㅋㅋㅋㅋ진짴ㅋㅋ
[민지이잉] [오전 12:02] 다들 잘쟈!!!
[윤희] [오전 12:04] 구랭
[윤희] [오전 12:04] 잘쟈"""

# 날짜 패턴
date_pattern = r"-+ (\d{4}년 \d{1,2}월 \d{1,2}일 [월화수목금토일]요일) -+"

# 메시지 패턴
message_pattern = r"\[([^\]]+)\] \[(오전|오후) (\d{1,2}:\d{2})\] (.+)"

# 날짜 찾기
dates = re.findall(date_pattern, text)

# 메시지 찾기
matches = re.finditer(message_pattern, text)

current_date_index = 0  # 처음에는 첫 번째 날짜부터 시작

for match in matches:
    name = match.group(1)
    am_pm = match.group(2)
    time = match.group(3)
    content = match.group(4)

    # 날짜가 바뀌면 current_date_index를 업데이트하여 새로운 날짜를 출력
    while current_date_index < len(dates) - 1 and match.start() > text.find(dates[current_date_index + 1]):
        current_date_index += 1

    current_date = dates[current_date_index] if current_date_index >= 0 else "날짜 없음"

    print(f"날짜: {current_date}")
    print(f"사람: {name}")
    print(f"시간: {am_pm} {time}")
    print(f"내용: {content}")
    print('-' * 40)


날짜: 2022년 1월 16일 일요일
사람: 민지이잉
시간: 오전 12:02
내용: ㅋㅋㅋㅋㅋㅋㅋㅋㄱㅇㄱㅋㅋㅋㅋㅋ진짴ㅋㅋ
----------------------------------------
날짜: 2022년 1월 16일 일요일
사람: 민지이잉
시간: 오전 12:02
내용: 다들 잘쟈!!!
----------------------------------------
날짜: 2022년 1월 16일 일요일
사람: 윤희
시간: 오전 12:04
내용: 구랭
----------------------------------------
날짜: 2022년 1월 16일 일요일
사람: 윤희
시간: 오전 12:04
내용: 잘쟈
----------------------------------------


## 카카오톡 데이터 파싱하기

In [ ]:
import re
import pandas as pd

def katalk_msg_parse(file_path):
  my_katalk_data = list()
  katalk_msg_pattern = r"\[([^\]]+)\] \[(오전|오후) (\d{1,2}:\d{2})\] (.+)"
  date_info = r"-+ (\d{4}년 \d{1,2}월 \d{1,2}일 [월화수목금토일]요일) -+"
  current_date = None

  for line in open(file_path, encoding='utf-8'):
    date_match = re.match(date_info, line)
    if date_match:
      current_date = date_match.group(1)
      continue

    message_match = re.match(katalk_msg_pattern, line)
    if message_match:
      name = message_match.group(1)
      am_pm = message_match.group(2)
      time = message_match.group(3)
      content = message_match.group(4)

      if current_date:
        my_katalk_data.append({
            'date_time' : f"{current_date} {am_pm} {time}",
            # 'date' : current_date,
            # 'time' : f"{am_pm} {time}",
            'user_name' : name,
            'text' : content
        })

      elif len(my_katalk_data) > 0:
        my_katalk_data[-1]['text'] += "\n" + line.strip()

  my_katalk_df = pd.DataFrame(my_katalk_data)

  return my_katalk_df



## 데이터 프레임으로 만들기

In [ ]:
import os

f_path = '/content/drive/MyDrive/Katalk_data_analysis/KakaoTalk_data'
f_name = 'katalk_friend.txt'

df = katalk_msg_parse(os.path.join(f_path, f_name))
df


,date_time,user_name,text
0,2022년 1월 16일 일요일 오전 12:02,민지이잉,ㅋㅋㅋㅋㅋㅋㅋㅋㄱㅇㄱㅋㅋㅋㅋㅋ진짴ㅋㅋ
1,2022년 1월 16일 일요일 오전 12:02,민지이잉,다들 잘쟈!!!
2,2022년 1월 16일 일요일 오전 12:04,윤희,구랭
3,2022년 1월 16일 일요일 오전 12:04,윤희,잘쟈
4,2022년 1월 16일 일요일 오전 12:04,윤희,이모티콘
...,...,...,...
45980,2025년 3월 5일 수요일 오후 2:22,민지이잉,다같이 모여서 피씨방 가쟈><
45981,2025년 3월 5일 수요일 오후 2:41,윤희,이모티콘
45982,2025년 3월 5일 수요일 오후 2:42,민지이잉,이모티콘
45983,2025년 3월 5일 수요일 오후 2:48,윤희,이모티콘


# 02 데이터 병합하기

## 데이터 프레임 붙이기 (concat)

In [ ]:
import pandas as pd

df1 = pd.DataFrame({'A' : [3, 4, 5],
                    'B' : ['a', 'b', 'c']})

df2 = pd.DataFrame({'A' : [3, 6, 9, 1],
                    'B' : ['b', 'b', 'e', 'a']})

df3 = pd.DataFrame({ 'B' : ['a', 'c', 'b', 'a'],
                     'C' : [3, 6, 9, 1],
                     'D' : ['b', 'b', 'e', 'a']})

df2 = df2.rename(columns={'A':'C', 'B':'D'})
pd.concat([df1, df2], axis=1)

,A,B,C,D
0,3.0,a,3,b
1,4.0,b,6,b
2,5.0,c,9,e
3,NaN,NaN,1,a


## 데이터 프레임 병합하기 (merge)

In [ ]:
pd.merge(df1, df3, on='B')

,A,B,C,D
0,3,a,3,b
1,3,a,1,a
2,4,b,9,e
3,5,c,6,b


## 카카오톡 데이터 합치기

In [ ]:
import os

f_path = '/content/drive/MyDrive/Katalk_data_analysis/KakaoTalk_data'
files = {
    'katalk_friend.txt' : 'friend',
    'katalk_oh.txt' : 'oh'
}

chat_room_list = list()

for f_name in files:
  chat_room = katalk_msg_parse(os.path.join(f_path, f_name))
  chat_room['chat_room_name'] = files[f_name]
  chat_room_list.append(chat_room)

df = pd.concat(chat_room_list,ignore_index=True)
df

,date_time,user_name,text,chat_room_name
0,2022년 1월 16일 일요일 오전 12:02,민지이잉,ㅋㅋㅋㅋㅋㅋㅋㅋㄱㅇㄱㅋㅋㅋㅋㅋ진짴ㅋㅋ,friend
1,2022년 1월 16일 일요일 오전 12:02,민지이잉,다들 잘쟈!!!,friend
2,2022년 1월 16일 일요일 오전 12:04,윤희,구랭,friend
3,2022년 1월 16일 일요일 오전 12:04,윤희,잘쟈,friend
4,2022년 1월 16일 일요일 오전 12:04,윤희,이모티콘,friend
...,...,...,...,...
49991,2025년 3월 6일 목요일 오후 3:27,윤희,아앗,oh
49992,2025년 3월 6일 목요일 오후 3:27,윤희,집가서 확인하자,oh
49993,2025년 3월 6일 목요일 오후 3:27,윤희,ㅋㅋㅋㅋㅋㅋㅋㅋㅋ,oh
49994,2025년 3월 6일 목요일 오후 3:48,야림이잉,이모티콘,oh


# 03 카카오톡 시간, 보낸 사람 정보 전처리

## 날짜, 시간 정보 전치리

In [75]:
import os

f_path = '/content/drive/MyDrive/Katalk_data_analysis/KakaoTalk_data'
f_name = 'katalk_friend.txt'

df = katalk_msg_parse(os.path.join(f_path, f_name))

df['date_time'] = df['date_time'].str.replace('오전', 'AM')
df['date_time'] = df['date_time'].str.replace('오후', 'PM')

weekday_map = {
    '월요일': 'Mon',
    '화요일': 'Tue',
    '수요일': 'Wed',
    '목요일': 'Thu',
    '금요일': 'Fri',
    '토요일': 'Sat',
    '일요일': 'Sun'
}

for kor, eng in weekday_map.items():
    df['date_time'] = df['date_time'].str.replace(kor, eng)

df.date_time


,date_time
0,2022년 1월 16일 Sun AM 12:02
1,2022년 1월 16일 Sun AM 12:02
2,2022년 1월 16일 Sun AM 12:04
3,2022년 1월 16일 Sun AM 12:04
4,2022년 1월 16일 Sun AM 12:04
...,...
45980,2025년 3월 5일 Wed PM 2:22
45981,2025년 3월 5일 Wed PM 2:41
45982,2025년 3월 5일 Wed PM 2:42
45983,2025년 3월 5일 Wed PM 2:48


시간 포맷가이드
: 정규표현식과 다르게 시간정보를 표현

In [76]:
date_examples = [
        {"date_time": "2021. 07. 08.",          "format": "%Y. %m. %d."},
        {"date_time": "2021년 07월 08일 12:30",          "format": "%Y년 %m월 %d일 %H:%M"},
        {"date_time": "2021-04-03 12:30",       "format": "%Y-%m-%d %H:%M"},
        {"date_time": "2020-08-03 AM 11:32",    "format": "%Y-%m-%d %p %I:%M"},
        {"date_time": "11/12/2020 PM 07:10:20",    "format": "%m/%d/%Y %p %I:%M:%S"},
        {"date_time": "29/03/1981 19:10:20 Sat", "format": "%d/%m/%Y %H:%M:%S %a"},
        {"date_time": "Jan. 11, 2021 Sat",      "format": "%b. %d, %Y %a"},
        {"date_time": "Mar 8, 2016 2:12 PM",      "format": "%b %d, %Y %I:%M %p"},
        {"date_time": "Tuesday, March 8, 2016",      "format": "%A, %B %d, %Y"},
        {"date_time": "8 Mar 2016 at 02:11",      "format": "%d %b %Y at %H:%M"}
]

for ex in date_examples:
    datetime_result = pd.to_datetime(ex["date_time"], format=ex["format"])
    print("{} --> {}".format(ex["date_time"], datetime_result))



2021. 07. 08. --> 2021-07-08 00:00:00
2021년 07월 08일 12:30 --> 2021-07-08 12:30:00
2021-04-03 12:30 --> 2021-04-03 12:30:00
2020-08-03 AM 11:32 --> 2020-08-03 11:32:00
11/12/2020 PM 07:10:20 --> 2020-11-12 19:10:20
29/03/1981 19:10:20 Sat --> 1981-03-29 19:10:20
Jan. 11, 2021 Sat --> 2021-01-11 00:00:00
Mar 8, 2016 2:12 PM --> 2016-03-08 14:12:00
Tuesday, March 8, 2016 --> 2016-03-08 00:00:00
8 Mar 2016 at 02:11 --> 2016-03-08 02:11:00


나의 카카오톡 날짜 포맷

In [77]:
df['date_time'] = pd.to_datetime(df['date_time'], format='%Y년 %m월 %d일 %a %p %I:%M')

df.date_time

,date_time
0,2022-01-16 00:02:00
1,2022-01-16 00:02:00
2,2022-01-16 00:04:00
3,2022-01-16 00:04:00
4,2022-01-16 00:04:00
...,...
45980,2025-03-05 14:22:00
45981,2025-03-05 14:41:00
45982,2025-03-05 14:42:00
45983,2025-03-05 14:48:00


## 날짜 세분화하기

In [78]:
df['year'] = df['date_time'].dt.year
df['month'] = df['date_time'].dt.month
df['day'] = df['date_time'].dt.day
df['day_of_week'] = df['date_time'].dt.day_name()
df['hour'] = df['date_time'].dt.hour

df

,date_time,user_name,text,year,month,day,day_of_week,hour
0,2022-01-16 00:02:00,민지이잉,ㅋㅋㅋㅋㅋㅋㅋㅋㄱㅇㄱㅋㅋㅋㅋㅋ진짴ㅋㅋ,2022,1,16,Sunday,0
1,2022-01-16 00:02:00,민지이잉,다들 잘쟈!!!,2022,1,16,Sunday,0
2,2022-01-16 00:04:00,윤희,구랭,2022,1,16,Sunday,0
3,2022-01-16 00:04:00,윤희,잘쟈,2022,1,16,Sunday,0
4,2022-01-16 00:04:00,윤희,이모티콘,2022,1,16,Sunday,0
...,...,...,...,...,...,...,...,...
45980,2025-03-05 14:22:00,민지이잉,다같이 모여서 피씨방 가쟈><,2025,3,5,Wednesday,14
45981,2025-03-05 14:41:00,윤희,이모티콘,2025,3,5,Wednesday,14
45982,2025-03-05 14:42:00,민지이잉,이모티콘,2025,3,5,Wednesday,14
45983,2025-03-05 14:48:00,윤희,이모티콘,2025,3,5,Wednesday,14


## 특정 기간 데이터 가져오기

2025년 1월 데이터만 따로 떼어내는 작업

In [87]:
start_date = '2025-01-01'
end_date = '2025-01-31'

mask = (df['date_time'] >= start_date) & (df['date_time'] <= end_date)
df_feb = df.loc[mask]
df_feb

,date_time,user_name,text,year,month,day,day_of_week,hour
44630,2025-01-01 00:07:00,야림이잉,햅삐뉴이어,2025,1,1,Wednesday,0
44631,2025-01-01 00:07:00,야림이잉,다들 건강하자!!!!,2025,1,1,Wednesday,0
44632,2025-01-01 00:07:00,윤희,앜ㅋㅋㅋ머쓱타트,2025,1,1,Wednesday,0
44633,2025-01-01 00:07:00,윤희,다들 새해 복 많이 받아아아,2025,1,1,Wednesday,0
44634,2025-01-01 00:08:00,윤희,이모티콘,2025,1,1,Wednesday,0
...,...,...,...,...,...,...,...,...
45267,2025-01-16 16:25:00,민지이잉,억울하지도 않아,2025,1,16,Thursday,16
45268,2025-01-16 16:25:00,민지이잉,왜냐면 어차피 한국이여도,2025,1,16,Thursday,16
45269,2025-01-16 16:25:00,민지이잉,"난 수원이기에,,",2025,1,16,Thursday,16
45270,2025-01-16 16:25:00,민지이잉,ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ,2025,1,16,Thursday,16


## 보낸 사람 정보 전처리

In [88]:
df['user_name'].value_counts()

,count
user_name,
민지이잉,16911
윤희,15860
야림이잉,13214


# 04 카카오톡 메시지 정보 전처리

## 메시지 정보 전처리
###1. 문자열 정보 요약

In [90]:
df['msg_len'] = df['text'].str.len()
df['msg_word_count'] = df['text'].str.split().str.len()

df

,date_time,user_name,text,year,month,day,day_of_week,hour,msg_len,msg_word_count
0,2022-01-16 00:02:00,민지이잉,ㅋㅋㅋㅋㅋㅋㅋㅋㄱㅇㄱㅋㅋㅋㅋㅋ진짴ㅋㅋ,2022,1,16,Sunday,0,20,1
1,2022-01-16 00:02:00,민지이잉,다들 잘쟈!!!,2022,1,16,Sunday,0,8,2
2,2022-01-16 00:04:00,윤희,구랭,2022,1,16,Sunday,0,2,1
3,2022-01-16 00:04:00,윤희,잘쟈,2022,1,16,Sunday,0,2,1
4,2022-01-16 00:04:00,윤희,이모티콘,2022,1,16,Sunday,0,4,1
...,...,...,...,...,...,...,...,...,...,...
45980,2025-03-05 14:22:00,민지이잉,다같이 모여서 피씨방 가쟈><,2025,3,5,Wednesday,14,16,4
45981,2025-03-05 14:41:00,윤희,이모티콘,2025,3,5,Wednesday,14,4,1
45982,2025-03-05 14:42:00,민지이잉,이모티콘,2025,3,5,Wednesday,14,4,1
45983,2025-03-05 14:48:00,윤희,이모티콘,2025,3,5,Wednesday,14,4,1


###2. 사진, 동영상 정보 추출하기

In [94]:
audio_visual_text = '^동영상$|^사진$|^사진 [0-9]{1,2}장$'
mask = df['text'].str.contains(audio_visual_text)
df.loc[mask, 'audio_visual'] = 1
df.loc[~mask, 'audio_visual'] = 0

df.loc[mask, 'msg_len'] = 0
df.loc[mask, 'msg_word_count'] = 0
df[df['audio_visual']==1]

,date_time,user_name,text,year,month,day,day_of_week,hour,msg_len,msg_word_count,audio_visual
246,2022-01-18 23:33:00,야림이잉,사진,2022,1,18,Tuesday,23,0,0,1.0
247,2022-01-18 23:51:00,야림이잉,사진,2022,1,18,Tuesday,23,0,0,1.0
248,2022-01-18 23:55:00,민지이잉,사진,2022,1,18,Tuesday,23,0,0,1.0
249,2022-01-18 23:56:00,야림이잉,사진,2022,1,18,Tuesday,23,0,0,1.0
250,2022-01-18 23:58:00,야림이잉,사진,2022,1,18,Tuesday,23,0,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
45261,2025-01-16 16:05:00,윤희,사진,2025,1,16,Thursday,16,0,0,1.0
45365,2025-03-03 14:14:00,야림이잉,사진,2025,3,3,Monday,14,0,0,1.0
45375,2025-03-03 15:00:00,야림이잉,사진,2025,3,3,Monday,15,0,0,1.0
45460,2025-03-03 20:48:00,민지이잉,사진,2025,3,3,Monday,20,0,0,1.0
